### 中醫醫案症狀詞抽取
- model: CRF
- 資料集：主症與兼症合併
- 類別標註: OS
- 詞位標注: 無
- 拆句: 以句點拆分
- 斷詞: 有做
- 詞性標註: 使用中科院的
- 引用辭典: 沒有
- 標點: 沒去

In [1]:
import json
import nltk
import numpy as np
from sklearn.metrics import classification_report

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1

In [3]:
import pynlpir  # 引入依赖包
pynlpir.open()  # 打开分词器


filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    
    #原文
    case_content = cols[2]
    case_content = case_content.replace(" ","")
    case_content = case_content.replace('#',',')
    sentences = case_content.split("。")
    
    #摘要
    summary_str = cols[3].strip()
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
        
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    #原文編碼有問題，跳過
    if idx in [64,223,251,389,480,673,744,751,769,1422,1541,1853]:
            continue
    
    #症狀詞作斷詞
    ne_list = []
    for term in main_list:
        terms_list = pynlpir.segment(term)
        for t in terms_list:
            ne_list.append(t[0])
    
    #每個醫案拆句
    for sent in sentences:
        
        word_list = []
    
        txt_list = pynlpir.segment(sent)
        for txt in txt_list:
            if txt[0] in ne_list:
                e = (txt[0],txt[1],"S")
            else :
                e = (txt[0],txt[1],"O")
            word_list.append(e)

        docs.append(word_list)
    
print(len(docs))

part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: '

48356


In [4]:
### 計算平均句長
total_wc = 0
for doc in docs:
    total_wc += len(doc)
    
ave = total_wc / 48901
print(ave)

23.615774728533157


In [5]:
data = docs

In [6]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    
    if postag is None:
            postag = 'none'

    # Common features for all words
    features = [
        'bias',
        'word=' + word,
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]
        
    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        
        if postag1 is None:
            postag1 = 'none'
            
        features.extend([
            '-1:word=' + word1,
            '-1.word.isdigit=%s' % word.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        
        if postag1 is None:
            postag1 = 'none'
            
        features.extend([
            '+1:word=' + word1,
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [7]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
print(len(X_train))
print(len(X_test))

38684
9672


In [9]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 60180
Seconds required: 1.538

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 267644.533941
Feature norm: 1.000000
Error norm: 153338.507040
Active features: 59108
Line search trials: 1
Line search step: 0.000001
Seconds required for this iteration: 0.400

***** Iteration #2 *****
Loss: 253467.699374
Feature norm: 0.899426
Error norm: 118177.481788
Active features: 57850
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.208

***** Iteration #3 *****
Loss: 244803.405630
Feature norm: 0.591464
Error norm: 175346.217344
Active features: 43341
Line search trials: 1
Line search step: 1.000000
Seconds requir

***** Iteration #39 *****
Loss: 100726.899482
Feature norm: 100.313329
Error norm: 5668.955935
Active features: 33513
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.206

***** Iteration #40 *****
Loss: 99888.964308
Feature norm: 110.102037
Error norm: 3822.772602
Active features: 33017
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.206

***** Iteration #41 *****
Loss: 99024.950469
Feature norm: 123.102866
Error norm: 7314.409203
Active features: 32038
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.204

***** Iteration #42 *****
Loss: 98338.363364
Feature norm: 132.581155
Error norm: 4059.721668
Active features: 31605
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.206

***** Iteration #43 *****
Loss: 97626.613615
Feature norm: 144.150784
Error norm: 4946.536590
Active features: 31316
Line search trials: 1
Line search step: 1.

***** Iteration #82 *****
Loss: 92793.690675
Feature norm: 205.529574
Error norm: 1548.924281
Active features: 27206
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.205

***** Iteration #83 *****
Loss: 92780.795728
Feature norm: 205.572407
Error norm: 2508.701616
Active features: 27162
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.204

***** Iteration #84 *****
Loss: 92765.705005
Feature norm: 205.611724
Error norm: 1248.645435
Active features: 27133
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.212

***** Iteration #85 *****
Loss: 92753.040543
Feature norm: 205.656147
Error norm: 2088.508793
Active features: 27075
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.205

***** Iteration #86 *****
Loss: 92738.722041
Feature norm: 205.697802
Error norm: 1564.478132
Active features: 27032
Line search trials: 1
Line search step: 1.0

***** Iteration #122 *****
Loss: 92436.551584
Feature norm: 207.550170
Error norm: 1830.038778
Active features: 26131
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.204

***** Iteration #123 *****
Loss: 92432.624417
Feature norm: 207.508253
Error norm: 2781.883829
Active features: 26113
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.205

***** Iteration #124 *****
Loss: 92424.771288
Feature norm: 207.484143
Error norm: 1663.050393
Active features: 26099
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.204

***** Iteration #125 *****
Loss: 92419.609665
Feature norm: 207.442904
Error norm: 2270.750859
Active features: 26086
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.205

***** Iteration #126 *****
Loss: 92412.839565
Feature norm: 207.400241
Error norm: 1629.320096
Active features: 26067
Line search trials: 1
Line search step

***** Iteration #167 *****
Loss: 92206.726595
Feature norm: 204.522504
Error norm: 2255.068656
Active features: 25353
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.209

***** Iteration #168 *****
Loss: 92202.079658
Feature norm: 204.496671
Error norm: 1786.492745
Active features: 25346
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.205

***** Iteration #169 *****
Loss: 92197.723008
Feature norm: 204.449951
Error norm: 2347.293581
Active features: 25331
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.205

***** Iteration #170 *****
Loss: 92192.805424
Feature norm: 204.428521
Error norm: 1577.991122
Active features: 25328
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.204

***** Iteration #171 *****
Loss: 92188.185321
Feature norm: 204.398022
Error norm: 1955.167458
Active features: 25308
Line search trials: 1
Line search step

In [10]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')

y_pred_train = [tagger.tag(xseq) for xseq in X_train]
y_pred_test = [tagger.tag(xseq) for xseq in X_test]

In [11]:
# precision error 
for i,doc in enumerate(X_test):
    print(i,'---------')
    for j,word in enumerate(doc):
        w = word[1].split("=")[1]
        y1 = y_pred_test[i][j]
        y2 = y_test[i][j]
        if 'S' == y1 and y1 != y2:
            print(w,y1,y2)

0 ---------
时 S O
伴有 S O
倦怠 S O
乏力 S O
头晕 S O
耳鸣 S O
纳 S O
呆 S O
寐 S O
差 S O
1 ---------
2 ---------
右 S O
胁 S O
3 ---------
4 ---------
5 ---------
6 ---------
7 ---------
8 ---------
9 ---------
10 ---------
11 ---------
12 ---------
13 ---------
14 ---------
15 ---------
16 ---------
17 ---------
量 S O
少 S O
18 ---------
时 S O
或 S O
发热 S O
恶心 S O
呕吐 S O
19 ---------
20 ---------
21 ---------
22 ---------
23 ---------
部 S O
24 ---------
25 ---------
26 ---------
27 ---------
28 ---------
29 ---------
小腹 S O
疼痛 S O
30 ---------
31 ---------
32 ---------
33 ---------
34 ---------
35 ---------
36 ---------
37 ---------
38 ---------
肢 S O
39 ---------
40 ---------
色 S O
41 ---------
42 ---------
43 ---------
44 ---------
45 ---------
46 ---------
47 ---------
48 ---------
49 ---------
50 ---------
51 ---------
52 ---------
有 S O
欠安 S O
大便 S O
略 S O
干 S O
53 ---------
54 ---------
55 ---------
56 ---------
57 ---------
口 S O
干 S O
口渴 S O
头晕 S O
项背 S O
麻木 S O
头痛 S O
梦 S O
58 ---------
59 -

684 ---------
685 ---------
686 ---------
687 ---------
688 ---------
689 ---------
690 ---------
691 ---------
692 ---------
693 ---------
酸 S O
痛 S O
694 ---------
695 ---------
696 ---------
697 ---------
698 ---------
699 ---------
700 ---------
701 ---------
702 ---------
703 ---------
704 ---------
705 ---------
706 ---------
707 ---------
708 ---------
709 ---------
710 ---------
711 ---------
712 ---------
713 ---------
714 ---------
715 ---------
716 ---------
717 ---------
718 ---------
719 ---------
720 ---------
721 ---------
722 ---------
723 ---------
724 ---------
725 ---------
腹痛 S O
矢 S O
气 S O
726 ---------
727 ---------
疼痛 S O
728 ---------
729 ---------
难以 S O
入睡 S O
730 ---------
731 ---------
732 ---------
733 ---------
734 ---------
735 ---------
736 ---------
737 ---------
738 ---------
胸 S O
739 ---------
不 S O
740 ---------
741 ---------
742 ---------
咽喉 S O
肿 S O
痛 S O
塞 S O
发热 S O
743 ---------
744 ---------
745 ---------
746 ---------
关节 S O
疼痛 S O
747 ----

1272 ---------
1273 ---------
1274 ---------
1275 ---------
1276 ---------
1277 ---------
1278 ---------
1279 ---------
1280 ---------
月经 S O
1281 ---------
1282 ---------
1283 ---------
1284 ---------
1285 ---------
大便 S O
稀 S O
1286 ---------
不 S O
左 S O
颈 S O
部 S O
红肿 S O
不 S O
饮食 S O
1287 ---------
1288 ---------
1289 ---------
1290 ---------
嘈杂 S O
周身 S O
无力 S O
咽 S O
夜间 S O
1291 ---------
咳嗽 S O
咳嗽 S O
稍 S O
恶心 S O
呕吐 S O
腹痛 S O
1292 ---------
1293 ---------
得 S O
温则舒 S O
1294 ---------
1295 ---------
1296 ---------
1297 ---------
眠 S O
差 S O
1298 ---------
1299 ---------
心 S O
前 S O
区 S O
疼痛 S O
1300 ---------
1301 ---------
1302 ---------
下肢 S O
浮肿 S O
痞 S O
嘈杂 S O
1303 ---------
1304 ---------
1305 ---------
1306 ---------
1307 ---------
1308 ---------
1309 ---------
1310 ---------
1311 ---------
白 S O
不易 S O
咯 S O
1312 ---------
1313 ---------
1314 ---------
1315 ---------
1316 ---------
1317 ---------
小腿 S O
抽筋 S O
1318 ---------
胃脘 S O
隐痛 S O
矢 S O
1319 ---------
1320 -----

1873 ---------
1874 ---------
1875 ---------
1876 ---------
1877 ---------
1878 ---------
1879 ---------
1880 ---------
不 S O
1881 ---------
1882 ---------
1883 ---------
1884 ---------
1885 ---------
腹 S O
1886 ---------
1887 ---------
1888 ---------
1889 ---------
1890 ---------
1891 ---------
1892 ---------
1893 ---------
1894 ---------
1895 ---------
1896 ---------
1897 ---------
1898 ---------
1899 ---------
1900 ---------
1901 ---------
1902 ---------
1903 ---------
1904 ---------
1905 ---------
1906 ---------
1907 ---------
1908 ---------
1909 ---------
关节 S O
肿 S O
痛 S O
1910 ---------
1911 ---------
1912 ---------
闷 S O
1913 ---------
1914 ---------
1915 ---------
1916 ---------
1917 ---------
1918 ---------
1919 ---------
1920 ---------
1921 ---------
1922 ---------
1923 ---------
1924 ---------
怕 S O
冷 S O
口 S O
干 S O
1925 ---------
1926 ---------
1927 ---------
1928 ---------
1929 ---------
痰 S O
1930 ---------
1931 ---------
1932 ---------
1933 ---------
1934 ---------
心慌 

2454 ---------
2455 ---------
2456 ---------
2457 ---------
2458 ---------
2459 ---------
2460 ---------
2461 ---------
2462 ---------
2463 ---------
2464 ---------
2465 ---------
2466 ---------
2467 ---------
2468 ---------
间断 S O
发 S O
作伴 S O
2469 ---------
2470 ---------
刺 S O
2471 ---------
2472 ---------
2473 ---------
2474 ---------
部 S O
食 S O
后 S O
胀 S O
满 S O
2475 ---------
2476 ---------
周身 S O
烙 S O
手 S O
2477 ---------
2478 ---------
腹痛 S O
2479 ---------
2480 ---------
2481 ---------
2482 ---------
2483 ---------
2484 ---------
赤 S O
2485 ---------
2486 ---------
2487 ---------
烂 S O
2488 ---------
2489 ---------
2490 ---------
2491 ---------
2492 ---------
2493 ---------
2494 ---------
痛 S O
2495 ---------
2496 ---------
2497 ---------
2498 ---------
2499 ---------
大便 S O
痰 S O
涎 S O
2500 ---------
2501 ---------
2502 ---------
2503 ---------
2504 ---------
2505 ---------
2506 ---------
少 S O
腹 S O
小便 S O
频 S O
而 S O
难 S O
出 S O
胀 S O
痛 S O
2507 ---------
呕吐 S O
黏 S O
涎 S

...

恶 S O
寒 S O
7098 ---------
胁 S O
痛 S O
7099 ---------
7100 ---------
7101 ---------
7102 ---------
7103 ---------
胃脘 S O
胀 S O
胀 S O
7104 ---------
7105 ---------
7106 ---------
头晕 S O
耳鸣 S O
7107 ---------
7108 ---------
7109 ---------
7110 ---------
月经 S O
不 S O
多 S O
带 S O
多 S O
黄 S O
7111 ---------
7112 ---------
7113 ---------
7114 ---------
7115 ---------
多 S O
7116 ---------
7117 ---------
脘 S O
心悸 S O
寐 S O
差 S O
7118 ---------
频频 S O
思 S O
食 S O
7119 ---------
7120 ---------
7121 ---------
7122 ---------
寒 S O
凉 S O
胃脘 S O
疼痛 S O
7123 ---------
7124 ---------
7125 ---------
7126 ---------
7127 ---------
7128 ---------
7129 ---------
7130 ---------
7131 ---------
7132 ---------
有 S O
不 S O
不 S O
寐 S O
7133 ---------
7134 ---------
7135 ---------
7136 ---------
7137 ---------
7138 ---------
7139 ---------
7140 ---------
7141 ---------
7142 ---------
寐 S O
干渴 S O
饮水 S O
7143 ---------
7144 ---------
满 S O
7145 ---------
7146 ---------
7147 ---------
7148 ---------
7149 ---------


隐痛 S O
7625 ---------
7626 ---------
7627 ---------
7628 ---------
7629 ---------
7630 ---------
7631 ---------
7632 ---------
7633 ---------
7634 ---------
7635 ---------
7636 ---------
及 S O
7637 ---------
7638 ---------
7639 ---------
行 S O
腹痛 S O
7640 ---------
7641 ---------
鼻 S O
衄 S O
7642 ---------
7643 ---------
7644 ---------
7645 ---------
7646 ---------
月经 S O
7647 ---------
7648 ---------
7649 ---------
7650 ---------
小腹 S O
隐痛 S O
7651 ---------
7652 ---------
7653 ---------
7654 ---------
7655 ---------
7656 ---------
7657 ---------
7658 ---------
7659 ---------
7660 ---------
7661 ---------
7662 ---------
7663 ---------
7664 ---------
7665 ---------
7666 ---------
痛 S O
7667 ---------
7668 ---------
7669 ---------
7670 ---------
7671 ---------
7672 ---------
7673 ---------
浮肿 S O
7674 ---------
7675 ---------
夹 S O
7676 ---------
7677 ---------
7678 ---------
7679 ---------
7680 ---------
7681 ---------
7682 ---------
7683 ---------
目 S O
泪 S O
7684 ---------
7685 -----

痛 S O
面颊 S O
部 S O
肿胀 S O
面色 S O
8208 ---------
8209 ---------
头晕 S O
8210 ---------
8211 ---------
8212 ---------
8213 ---------
全身 S O
汗 S O
出 S O
汗 S O
出 S O
后 S O
恶 S O
风 S O
身体 S O
恶 S O
寒 S O
头痛 S O
8214 ---------
8215 ---------
8216 ---------
不 S O
8217 ---------
痛 S O
8218 ---------
8219 ---------
无力 S O
8220 ---------
8221 ---------
8222 ---------
8223 ---------
潜血 S O
时 S O
8224 ---------
8225 ---------
8226 ---------
8227 ---------
8228 ---------
8229 ---------
8230 ---------
8231 ---------
记忆力 S O
下降 S O
胃脘 S O
嘈杂 S O
8232 ---------
8233 ---------
8234 ---------
8235 ---------
8236 ---------
气 S O
8237 ---------
8238 ---------
8239 ---------
8240 ---------
8241 ---------
8242 ---------
8243 ---------
8244 ---------
8245 ---------
8246 ---------
8247 ---------
8248 ---------
8249 ---------
腰部 S O
8250 ---------
噫 S O
恶 S O
体 S O
倦 S O
便 S O
溏 S O
8251 ---------
8252 ---------
8253 ---------
8254 ---------
8255 ---------
8256 ---------
8257 ---------
8258 ---------
8259 -----

潮 S O
热 S O
出 S O
汗 S O
8729 ---------
8730 ---------
8731 ---------
8732 ---------
8733 ---------
8734 ---------
8735 ---------
8736 ---------
8737 ---------
8738 ---------
8739 ---------
8740 ---------
8741 ---------
8742 ---------
8743 ---------
8744 ---------
8745 ---------
8746 ---------
左侧 S O
8747 ---------
8748 ---------
伤口 S O
8749 ---------
8750 ---------
8751 ---------
8752 ---------
8753 ---------
8754 ---------
8755 ---------
面色 S O
苍白 S O
眼眶 S O
下 S O
8756 ---------
8757 ---------
8758 ---------
8759 ---------
8760 ---------
伴 S O
8761 ---------
8762 ---------
8763 ---------
失眠 S O
8764 ---------
不 S O
8765 ---------
麻木 S O
8766 ---------
8767 ---------
8768 ---------
8769 ---------
8770 ---------
8771 ---------
8772 ---------
8773 ---------
8774 ---------
8775 ---------
8776 ---------
8777 ---------
8778 ---------
痛 S O
丧失 S O
坐 S O
数 S O
小时 S O
不 S O
动 S O
8779 ---------
尿血 S O
血 S O
8780 ---------
胃 S O
8781 ---------
8782 ---------
8783 ---------
8784 ---------
8785 -

9280 ---------
9281 ---------
9282 ---------
9283 ---------
9284 ---------
9285 ---------
9286 ---------
9287 ---------
9288 ---------
腹泻 S O
9289 ---------
9290 ---------
9291 ---------
9292 ---------
9293 ---------
9294 ---------
9295 ---------
9296 ---------
9297 ---------
心慌 S O
鼻 S O
衄 S O
9298 ---------
痰 S O
9299 ---------
9300 ---------
9301 ---------
9302 ---------
9303 ---------
9304 ---------
9305 ---------
嗜睡 S O
欲 S O
饮食 S O
9306 ---------
尿 S O
量 S O
9307 ---------
9308 ---------
9309 ---------
9310 ---------
9311 ---------
不安 S O
夜 S O
则 S O
9312 ---------
心烦 S O
寐 S O
汗 S O
出 S O
喜 S O
悲 S O
欲 S O
哭 S O
手足 S O
心 S O
热 S O
痛 S O
9313 ---------
9314 ---------
9315 ---------
9316 ---------
9317 ---------
9318 ---------
9319 ---------
头 S O
痛 S O
9320 ---------
9321 ---------
黏液 S O
9322 ---------
9323 ---------
9324 ---------
9325 ---------
9326 ---------
9327 ---------
9328 ---------
形体 S O
羸 S O
不 S O
馨 S O
9329 ---------
9330 ---------
9331 ---------
9332 ---------
9333

In [12]:
# Create a mapping of labels to indices
labels = {"O": 0,"S": 1}

# training data F1-score
predictions_train = np.array([labels[tag] for row in y_pred_train for tag in row])
truths_train = np.array([labels[tag] for row in y_train for tag in row])
print("F1 score for training set:\n")
print(classification_report(
    truths_train, predictions_train,
    target_names=["O", "S"]))

print("\n")

# test data F1-score
predictions_test = np.array([labels[tag] for row in y_pred_test for tag in row])
truths_test = np.array([labels[tag] for row in y_test for tag in row])
print("F1 score for test set:\n")
print(classification_report(
    truths_test, predictions_test,
    target_names=["O", "S"]))

F1 score for training set:

             precision    recall  f1-score   support

          O       0.97      0.98      0.97    851330
          S       0.72      0.57      0.64     69776

avg / total       0.95      0.95      0.95    921106



F1 score for test set:

             precision    recall  f1-score   support

          O       0.96      0.98      0.97    216777
          S       0.65      0.53      0.58     16952

avg / total       0.94      0.95      0.94    233729

